## 1. Install libraries

In [2]:
!pip install --quiet pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 torchtext==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --quiet transformers
#==4.16.2
!pip install --quiet pytorch-lightning 
#==1.2.2 
!pip install --quiet tokenizers
#==0.9.4 
!pip install --quiet sentencepiece
#==0.1.94 
!pip install --quiet tqdm
#==4.56.0 

     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:24tcmalloc: large alloc 1147494400 bytes == 0x559e3f514000 @  0x7f6973b69615 0x559e06b9e3bc 0x559e06c7f18a 0x559e06ba11cd 0x559e06c93b3d 0x559e06c15458 0x559e06c1002f 0x559e06ba2aba 0x559e06c152c0 0x559e06c1002f 0x559e06ba2aba 0x559e06c11cd4 0x559e06c94986 0x559e06c11350 0x559e06c94986 0x559e06c11350 0x559e06c94986 0x559e06c11350 0x559e06ba2f19 0x559e06be6a79 0x559e06ba1b32 0x559e06c151dd 0x559e06c1002f 0x559e06ba2aba 0x559e06c11cd4 0x559e06c1002f 0x559e06ba2aba 0x559e06c10eae 0x559e06ba29da 0x559e06c11108 0x559e06c1002f
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:37tcmalloc: large alloc 1434370048 bytes == 0x559e83b6a000 @  0x7f6973b69615 0x559e06b9e3bc 0x559e06c7f18a 0x559e06ba11cd 0x559e06c93b3d 0x559e06c15458 0x559e06c1002f 0x559e06ba2aba 0x559e06c152c0 0x559e06c1002f 0x559e06ba2aba 0x559e06c11cd4 0x559e06c94986 0x559e06c11350 0x559e06c94986 0x559e06c11350 0x559e06c94986 0x559e06c11350 0x559e

In [3]:
# restart runtime
# import os

# def restart_runtime():
#   os.kill(os.getpid(), 9)

# restart_runtime()

In [4]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Fri Feb 25 02:26:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# GPT-3 load


In [5]:
#@title GPT3-kakaobrain version
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM , PreTrainedTokenizerFast , GPTJForCausalLM

tokenizer = PreTrainedTokenizerFast.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)
model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(64512, 4096)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=

# Generate

In [6]:
import pandas as pd
import numpy as np
PATH = "/content/drive/MyDrive/Colab Notebooks/data/chatbot/"
datasets = pd.read_excel(PATH+"BDI,HAM-D 문항.xlsx", sheet_name="BDI_more")

In [7]:
def gen_reply(prompt):
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
        gen_tokens = model.generate(tokens, do_sample=True, temperature=0.88, max_length=len(prompt)/1.2)
        generated = tokenizer.batch_decode(gen_tokens)[0]
        return generated[len(prompt):].split("\n")[0]

In [8]:
bdi_dict = {}
for bdi in datasets['항목'].unique():
    bdi_dict[bdi]= []

for index, row in datasets.iterrows():
    if type(row["질문형"])==float:
        print(index)
        break 
    bdi_dict[row["항목"]] += [f'{row["항목"]}:{row["질문"]} => {row["항목"]}:{row["질문형"]}']

84


In [ ]:
#@title Make Data
#@markdown 5000여개 reply 추가하는데 colab으로 작업시간 3일 정도?
from tqdm.notebook import tqdm
for index, row in tqdm(datasets.iterrows(),
                       f"Processing Data with GPT-3", 
                       total=len(datasets),
                       leave=True):
    if type(row["질문형"])==float:
        prompt="\n".join([np.random.choice(bdi_dict[bdi]) for bdi in bdi_dict]) +\
               "\n" + f'{row["항목"]}:{row["질문"]} =>'
        row["질문형"]=gen_reply(prompt)
    if index%100 ==0:
        datasets.to_excel(PATH+"BDI,HAM-D 문항 GPT.xlsx",index=False,)
datasets.to_excel(PATH+"BDI,HAM-D 문항 GPT.xlsx",index=False,)

Processing Data with GPT-3:   0%|          | 0/1008 [00:00<?, ?it/s]